In [ ]:
import numpy as np
from os.path import join
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import hydromt

## read flood data

In [ ]:
mdir = r"../../3_models/sfincs"
rdir = r"../../4_results"

In [ ]:
index_cols = ['h_rp','p_rp','qb_rp','qp_rp']
ds = xr.open_zarr(join(rdir, 'hmax.zarr'))#.swap_dims({'index':'scen'}).set_coords(index_cols)
da1 = ds['hmax']
nodata = da1.raster.nodata
print(nodata)
da1

## Univariate and full dependence / indepence scenarios

In [ ]:
# save bias corrected floodmaps
for rp in  np.array([2,5,10,25,50,100,250,500], dtype=int):
    scens = {
        f'qb{rp:03d}_qp000_h000_p000': f'qb_rp{rp:03d}', 
        f'qb000_qp{rp:03d}_h000_p000': f'qp_rp{rp:03d}', 
        f'qb000_qp000_h{rp:03d}_p000': f'h_rp{rp:03d}', 
        f'qb000_qp000_h000_p{rp:03d}': f'p_rp{rp:03d}', 
        f'qb{rp:03d}_qp{rp:03d}_h{rp:03d}_p{rp:03d}': f'compound_fulldep_rp{rp:03d}'
    }
    ds_scens = xr.merge([da1.sel(scen=scen).rename(name).reset_coords(drop=True) for scen, name in scens.items()])
    ds_scens[f'compound_indep_rp{rp:03d}'] = da1.sel(scen=list(scens.keys())[:-1]).max('scen')
    ds_scens.raster.set_crs(da1.raster.crs)
    for dvar in ds_scens.data_vars.keys():
        ds_scens[dvar].raster.set_nodata(da1.raster.nodata)
    ds_scens.raster.flipud().raster.to_mapstack(join(rdir, 'hmax'))

## Calculate compound comound flood depths

In [ ]:

df_sample = pd.read_csv(join(rdir, r'sim_EVENTS_rp.csv'), index_col=0).rename(columns={'h_tsw_rp': 'h_rp', 'h_tsw': 'h'})


df_flood = da1.reset_coords().drop(['band', 'spatial_ref', 'x', 'y', 'hmax'], errors='ignore').to_dataframe()
# df_flood[index_cols] = np.maximum(1, df_flood[index_cols])
ds_flood_rp = df_flood.reset_index().drop(columns=index_cols).set_index(
    pd.MultiIndex.from_frame(df_flood[index_cols])
).to_xarray()
ds_flood_rp['index'] = ds_flood_rp['index'].fillna(-1).astype(int)
ds_flood_rp


In [ ]:
# get nearest scen for each sample based on rp
# TODO: should this be based on q or log(rp) ?
# NOTE this is an approximation!
ds0_rp = df_sample[index_cols + ['year']].to_xarray().set_coords('year')
df_sample['index'] = ds_flood_rp['index'].sel(ds0_rp, method='nearest')
df_sample['scen'] = df_flood.reset_index().loc[df_sample['index'].values, 'scen'].values

In [ ]:
_scen_unique, idxs_reverse = np.unique(df_sample['scen'], return_inverse=True)
print(_scen_unique.size)
flddph = da1.sel(scen=_scen_unique).load()

In [ ]:
# this cell takes some time, approx ~30 min
# it can also be done much more intellegent accounting for the weight of each sim event based on number of occurrences
# 0 2 5 10 25 50 100 250 500
rps=np.array([2, 5, 10, 25, 50, 100, 250, 500])
n = df_sample.year.max()
cdf = np.arange(n)/n
bins = df_sample[['year']].values.flatten()
def _fldsrt(fld0, idxs_reverse=idxs_reverse, rp=rps, bins=bins, cdf=cdf):
    if np.all(fld0==0):
        return 0.0
    fld0 = fld0[idxs_reverse] # explode unique simulations to sample size
    df0 = pd.DataFrame(data={'fld0': fld0, 'bin':bins})
    fld_max = df0[['fld0', 'bin']].groupby('bin').max().values.flatten() # get AM
    fld_sorted = np.sort(fld_max) # sort AM
    return np.interp(1-1/rp, cdf, fld_sorted)
data=np.apply_along_axis(_fldsrt, flddph.get_axis_num("scen"), np.maximum(0, flddph))

In [ ]:
nodata=-999
da_cmpnd = xr.DataArray(
    data=data, 
    dims=('rp', 'y', 'x'),
    coords={'rp':rps, **da1.raster.coords}
).where(flddph.isel(scen=0)!=nodata, nodata)
da_cmpnd.raster.set_nodata(nodata)
da_cmpnd.raster.set_crs(da1.raster.crs)
for rp0 in rps:
    da_cmpnd.sel(rp=rp0).raster.to_raster(join(rdir, 'hmax', f'compound_obsdep_rp{rp0:03d}.tif'), compress='lzw')

## plots 

In [ ]:
# TODO